In [1]:
import os
import pickle
import numpy
import random 

from music21 import note, chord
from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

2021-12-02 10:22:41.703161: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-02 10:22:41.703178: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from music21 import corpus, converter
from IPython.display import Image, Audio
def play(music):
  filename = music.write('mid')
  !fluidsynth -ni font.sf2 $filename -F $filename\.wav -r 16000 > /dev/null
  display(Audio(str(filename) + '.wav'))

In [5]:
BASE_DIR = os.getcwd()
music_folder = 'classical' 
music_name = 'classical'

run_folder = os.path.join(BASE_DIR,"output",music_folder)
store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join(BASE_DIR,'data', music_name)

if not os.path.exists(run_folder):
    os.makedirs(run_folder)
    os.makedirs(os.path.join(run_folder, 'store'))
    os.makedirs(os.path.join(run_folder, 'output'))
    os.makedirs(os.path.join(run_folder, 'weights'))
    os.makedirs(os.path.join(run_folder, 'viz'))
    
mode = 'load' # 'load' # 

# data params
intervals = range(1)
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True

In [6]:
import random
#Tranforma los datos a un formato más estructurado, separando la pista en los silencios, los acordes, las notas, etc.
songs = []
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []
    test = 1
    for i in random.sample(range(0, len(music_list)), 125):
        songs.append(os.path.basename(music_list[i]))
        try:
            if test%5==0:
                print('-'*20,test,'-'*20)
            #Desconpone la pista en acordes 
            original_score = parser.parse(music_list[i]).chordify()
            
            for interval in intervals:
                score = original_score.transpose(interval)
                notes.extend(['START'] * seq_len)
                durations.extend([0]* seq_len)
                #Se verifica que tipo de entrada es (silencio, acorde, nota) y se almacena junto con su duration en dos lista (notes, durations)
                for element in score.flat:
                    
                    if isinstance(element, note.Note):
                        if element.isRest:
                            notes.append(str(element.name))
                            durations.append(element.duration.quarterLength)
                        else:
                            notes.append(str(element.nameWithOctave))
                            durations.append(element.duration.quarterLength)

                    if isinstance(element, chord.Chord):
                        notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                        durations.append(element.duration.quarterLength)
            test+=1
        except:
            print('error in file: %s' % file)
            os.remove(file)
    #Crearemos o sobreescribiremos los archivos donde se almacenaras las notas y duraciones
    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
    with open(os.path.join(store_folder, 'train.txt'), 'w') as f:
        for s in songs:
            f.write(str(s) + '\n')
            
#En el caso de que no nos encontremos en modo "build", se cargarán los datos previamente almacenados
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f)

In [8]:
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [9]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [10]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)

2021-11-24 14:24:45.375632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-24 14:24:45.375702: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-24 14:24:45.375754: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a0da3121b20d): /proc/driver/nvidia/version does not exist
2021-11-24 14:24:45.376274: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.9/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: 

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1,
    checkpoint2
   #,early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=150, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )

2021-11-24 14:25:24.997121: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 28536758800 exceeds 10% of free system memory.
2021-11-24 14:25:34.839126: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-24 14:25:34.856415: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3000000000 Hz


Epoch 1/150
4404/4404 [==============================] - ETA: 0s - loss: 10.7159 - pitch_loss: 9.4939 - duration_loss: 1.2220

2021-11-24 14:33:22.994514: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 7134392232 exceeds 10% of free system memory.


4404/4404 [==============================] - 494s 112ms/step - loss: 10.7159 - pitch_loss: 9.4939 - duration_loss: 1.2220 - val_loss: 10.9319 - val_pitch_loss: 9.8399 - val_duration_loss: 1.0920


/usr/local/lib/python3.9/dist-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/150
4404/4404 [==============================] - 491s 111ms/step - loss: 10.2790 - pitch_loss: 9.2411 - duration_loss: 1.0379 - val_loss: 10.8235 - val_pitch_loss: 9.7661 - val_duration_loss: 1.0574
Epoch 3/150
4404/4404 [==============================] - 491s 112ms/step - loss: 10.2355 - pitch_loss: 9.2976 - duration_loss: 0.9379 - val_loss: 11.4330 - val_pitch_loss: 10.3713 - val_duration_loss: 1.0617
Epoch 4/150
4404/4404 [==============================] - 491s 112ms/step - loss: 10.9497 - pitch_loss: 10.1065 - duration_loss: 0.8432 - val_loss: 12.2786 - val_pitch_loss: 11.1938 - val_duration_loss: 1.0848
Epoch 5/150
4404/4404 [==============================] - 491s 112ms/step - loss: 11.1553 - pitch_loss: 10.3699 - duration_loss: 0.7853 - val_loss: 12.6363 - val_pitch_loss: 11.4116 - val_duration_loss: 1.2246
Epoch 6/150
4404/4404 [==============================] - 492s 112ms/step - loss: 11.1975 - pitch_loss: 10.4558 - duration_loss: 0.7417 - val_loss: 12.9362 - val_pitch_